In [1]:
pip install beautifulsoup4 selenium


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium --upgrade


Note: you may need to restart the kernel to use updated packages.


In [67]:
import requests

In [14]:
import time
import threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException

MAX_RETRIES = 3

def extract_links(driver):
    links = []

    max_scrolls = 20
    scrolls = 0

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
    time.sleep(20)

    while True:
        try:
            house_links_elements = driver.find_elements(By.CSS_SELECTOR, "a.ember-view.listing-tile-info")

            for link in house_links_elements:
                link_href = link.get_attribute('href')
                if link_href not in links:
                    links.append(link_href)

            if len(links) >= 10:
                break

            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(3)

            scrolls += 1
            if scrolls >= max_scrolls:
                break

        except WebDriverException:
            return []

    return links

def extract_links_from_pages(driver, start_page, end_page):
    all_links = []
    base_url = 'https://www.realestate.co.nz/residential/rental/auckland/auckland-city?by=oldest'
    driver.execute_script("document.body.style.zoom = '0.5'")
    for i in range(start_page, end_page + 1):
        retries = 0
        success = False
        while retries < MAX_RETRIES and not success:
            try:
                if i > 1:
                    url = f"{base_url}&page={i}"
                else:
                    url = base_url
                driver.get(url)
                page_links = extract_links(driver)
                all_links.extend(page_links)
                print(f"Extracted {len(page_links)} links from page {i}")
                success = True
            except WebDriverException:
                retries += 1
                print(f"Error loading page {i}. Retrying ({retries}/{MAX_RETRIES})...")
                if retries == MAX_RETRIES:
                    print(f"Failed to load page {i} after {MAX_RETRIES} retries. Moving to next page.")

    return all_links

def start_extraction_thread(start_page, end_page):
    options = webdriver.ChromeOptions()
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.popups": 2
    }
    options.add_experimental_option("prefs", prefs)
    options.add_argument("--disable-popup-blocking")

    while start_page <= end_page:
        try:
            driver = webdriver.Chrome(options=options)
            driver.set_script_timeout(300)
            links = extract_links_from_pages(driver, start_page, end_page)
            for link in links:
                print(link)
            driver.quit()
            break  # Exit the loop if extraction was successful
        except WebDriverException:
            print("Driver crashed. Restarting...")
            start_page = end_page  # Adjust the start_page if needed

if __name__ == "__main__":
    t1 = threading.Thread(target=start_extraction_thread, args=(1, 20))
    t2 = threading.Thread(target=start_extraction_thread, args=(21, 40))

    t1.start()
    t2.start()

    t1.join()
    t2.join()



Extracted 13 links from page 21
Extracted 11 links from page 1
Extracted 11 links from page 22
Extracted 10 links from page 2
Extracted 10 links from page 3
Extracted 10 links from page 23
Extracted 15 links from page 4
Extracted 13 links from page 24
Extracted 12 links from page 5
Error loading page 25. Retrying (1/3)...
Error loading page 6. Retrying (1/3)...
Error loading page 25. Retrying (2/3)...
Extracted 12 links from page 6
Error loading page 25. Retrying (3/3)...
Failed to load page 25 after 3 retries. Moving to next page.
Extracted 17 links from page 7
Extracted 12 links from page 26
Error loading page 8. Retrying (1/3)...
Error loading page 27. Retrying (1/3)...
Extracted 10 links from page 8
Error loading page 27. Retrying (2/3)...
Extracted 10 links from page 9
Extracted 10 links from page 10
Extracted 10 links from page 27
Extracted 10 links from page 11
Extracted 10 links from page 28
Extracted 11 links from page 29
Extracted 10 links from page 12
Extracted 10 links from

Extracted 10 links from page 38
Extracted 10 links from page 39
Extracted 5 links from page 40
https://www.realestate.co.nz/42435003/residential/rent/2011-11-liverpool-street-auckland-central
https://www.realestate.co.nz/42435098/residential/rent/502-great-north-road-grey-lynn
https://www.realestate.co.nz/42435158/residential/rent/6-magma-crescent-stonefields
https://www.realestate.co.nz/42435172/residential/rent/811-76-wakefield-street-auckland-central
https://www.realestate.co.nz/42435177/residential/rent/30a-dromorne-road-remuera
https://www.realestate.co.nz/42435216/residential/rent/3-365-great-south-road-ellerslie
https://www.realestate.co.nz/42435227/residential/rent/3d-31-scanlan-street-grey-lynn
https://www.realestate.co.nz/42435257/residential/rent/5a-pasteur-place-new-windsor
https://www.realestate.co.nz/42435288/residential/rent/24-paku-lane-mount-wellington
https://www.realestate.co.nz/42435365/residential/rent/a-23-watea-road-sandringham
https://www.realestate.co.nz/424353

In [68]:
url = 'https://raw.githubusercontent.com/robertoaltran/Population/main/links.txt'
response = requests.get(url)
sites = response.text.splitlines()

# Remover qualquer espaço em branco ou caracteres de nova linha no final de cada linha
sites = [site.strip() for site in sites]


In [71]:
sites

['https://www.realestate.co.nz/42435003/residential/rent/2011-11-liverpool-street-auckland-central',
 'https://www.realestate.co.nz/42435098/residential/rent/502-great-north-road-grey-lynn',
 'https://www.realestate.co.nz/42435158/residential/rent/6-magma-crescent-stonefields',
 'https://www.realestate.co.nz/42435172/residential/rent/811-76-wakefield-street-auckland-central',
 'https://www.realestate.co.nz/42435177/residential/rent/30a-dromorne-road-remuera',
 'https://www.realestate.co.nz/42435216/residential/rent/3-365-great-south-road-ellerslie',
 'https://www.realestate.co.nz/42435227/residential/rent/3d-31-scanlan-street-grey-lynn',
 'https://www.realestate.co.nz/42435257/residential/rent/5a-pasteur-place-new-windsor',
 'https://www.realestate.co.nz/42435288/residential/rent/24-paku-lane-mount-wellington',
 'https://www.realestate.co.nz/42435365/residential/rent/a-23-watea-road-sandringham',
 'https://www.realestate.co.nz/42435392/residential/rent/26-old-mill-road-grey-lynn',
 'ht

In [72]:
quantity_elements = len(sites)
print("Elements:", quantity_elements)

Elements: 405


In [73]:
sites.sort()

In [43]:
sites = [url for url in sites if not url.endswith('rent')]
sites.remove('Extracted 10 links from page 20')


In [46]:
siteurls = sites[:]

In [48]:
import threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd

MAX_RETRIES = 2

def initialize_driver():
    options = webdriver.ChromeOptions()
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.popups": 2
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver

def extract_info_chunk(start_idx, end_idx, driver, info_chunk_list):
    for i in range(start_idx, end_idx):
        url = sites[i]
        info_dict = extract_info((url, driver))
        info_chunk_list.append(info_dict)
        print(info_dict)  

def extract_info(args):
    url, driver = args
    info_dict = {}
    
    retries = 0
    while retries < MAX_RETRIES:
        try:
            driver.get(url)
            print(f"Scraping: {url}")

            address_element = driver.find_element(By.CSS_SELECTOR, "h1[data-test='listing-title']")
            address = address_element.text.strip()
            info_dict["Address"] = address

            price_element = driver.find_element(By.CSS_SELECTOR, "h3.text-xl.font-semibold")
            price = price_element.text.strip()
            info_dict["Price"] = price

            elements = driver.find_elements(By.CSS_SELECTOR, "span.leading-\\[-1\\]")
            max_elements_to_extract = 8
            for i, element in enumerate(elements, start=1):
                if i > max_elements_to_extract:
                    break
                text = element.text.strip()
                info_dict[f"Info_{i}"] = text

            if len(info_dict) >= 6:
                break

        except (NoSuchElementException, TimeoutException) as e:
            retries += 1
            if retries == MAX_RETRIES:
                info_dict["Error"] = str(e)
                break

    return info_dict

if __name__ == "__main__":
   
    drivers = [initialize_driver() for _ in range(3)]
   
    info_list1 = []
    info_list2 = []
    info_list3 = []

    third_length = len(siteurls) // 3

    t1 = threading.Thread(target=extract_info_chunk, args=(0, third_length, drivers[0], info_list1))
    t2 = threading.Thread(target=extract_info_chunk, args=(third_length, 2 * third_length, drivers[1], info_list2))
    t3 = threading.Thread(target=extract_info_chunk, args=(2 * third_length, len(sites), drivers[2], info_list3))

    # iniciating threads
    t1.start()
    t2.start()
    t3.start()

    t1.join()
    t2.join()
    t3.join()

    # joinning results
    info_list = info_list1 + info_list2 + info_list3

    # closing drivers
    for driver in drivers:
        driver.quit()

    df = pd.DataFrame(info_list)
    df.to_csv('scraped_data.csv', index=False)
    print("\nScraping results:")
    print(df)



Scraping: https://www.realestate.co.nz/42438043/residential/rent/11c-athens-road-one-tree-hill
Scraping: https://www.realestate.co.nz/42426231/residential/rent/2-1976-great-north-road-avondale
Scraping: https://www.realestate.co.nz/3745538/residential/rent/20-9a-esplanade-road-mount-eden
{'Address': '20/9A Esplanade Road, Mount Eden, Auckland City', 'Price': '$360 per week', 'Info_1': 'Unit', 'Info_2': '1', 'Info_3': '1', 'Info_4': '1', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Address': '11C Athens Road, One Tree Hill, Auckland City', 'Price': '$1,000 per week', 'Info_1': 'House', 'Info_2': '4', 'Info_3': '3', 'Info_4': '1', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Address': '2/1976 Great North Road, Avondale, Auckland City', 'Price': '$590 per week', 'Info_1': 'Apartment', 'Info_2': '2', 'Info_3': '1', 'Info_4': '', 'Info_5': '', 'Info_6': '', 'Info_7': '1 storey'}
Scraping: https://www.realestate.co.nz/42187622/residential/rent/46-montrose-street-poi

Scraping: https://www.realestate.co.nz/42438076/residential/rent/108-223a-greenlane-west-epsom
Scraping: https://www.realestate.co.nz/42427859/residential/rent/7-elam-street-parnell
{'Address': '108/223a Greenlane West , Epsom, Auckland City', 'Price': '$770 per week', 'Info_1': 'Apartment', 'Info_2': '2', 'Info_3': '2', 'Info_4': '1', 'Info_5': '1 ensuite', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
Scraping: https://www.realestate.co.nz/42214955/residential/rent/4g-16-market-place-viaduct-auckland-central
Scraping: https://www.realestate.co.nz/42438319/residential/rent/6-52-ascot-avenue-remuera
{'Address': '7 Elam Street, Parnell, Auckland City', 'Price': '$1,895 per week', 'Info_1': 'House', 'Info_2': '3', 'Info_3': '3', 'Info_4': '265m2', 'Info_5': '2', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Address': '6/52 Ascot Avenue, Remuera, Auckland City', 'Price': '$690 per week', 'Info_1': 'Townhouse', 'Info_2': '3', 'Info_3': '1', 'Info_4': '2', 'Info_5': '2', 'Info_6': '', 'Info_7': '

Scraping: https://www.realestate.co.nz/42428693/residential/rent/65-selwyn-avenue-mission-bay
Scraping: https://www.realestate.co.nz/42438709/residential/rent/37-tamora-lane-avondale
{'Address': '65 Selwyn Avenue, Mission Bay, Auckland City', 'Price': '$795 per week', 'Info_1': 'House', 'Info_2': '3', 'Info_3': '2', 'Info_4': '2', 'Info_5': 'Non furnished', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Address': '37 Tamora Lane, Avondale, Auckland City', 'Price': '$930 per week', 'Info_1': 'Townhouse', 'Info_2': '4', 'Info_3': '2', 'Info_4': '1', 'Info_5': '1', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Error': 'Message: timeout: Timed out receiving message from renderer: 288.820\n  (Session info: chrome=118.0.5993.70)\nStacktrace:\n0   chromedriver                        0x0000000100864510 chromedriver + 4310288\n1   chromedriver                        0x000000010085c4bc chromedriver + 4277436\n2   chromedriver                        0x000000010048fb6c chromedriver + 293740\n3   chrome

Scraping: https://www.realestate.co.nz/42439020/residential/rent/83-empire-road-epsom
{'Address': '10 Green Lane East, Remuera, Auckland City', 'Price': '$1,250 per week', 'Info_1': 'House', 'Info_2': '5', 'Info_3': '2', 'Info_4': '2', 'Info_5': '4', 'Info_6': 'Non furnished', 'Info_7': '', 'Info_8': ''}
{'Address': '83 Empire Road, Epsom, Auckland City', 'Price': '$1,750 per week', 'Info_1': 'House', 'Info_2': '5', 'Info_3': '1', 'Info_4': '2', 'Info_5': '2', 'Info_6': 'Non furnished', 'Info_7': '2 ensuites', 'Info_8': ''}
Scraping: https://www.realestate.co.nz/42430172/residential/rent/23-paget-street-freemans-bay
Scraping: https://www.realestate.co.nz/42439030/residential/rent/3-6-sarsfield-street-herne-bay
Scraping: https://www.realestate.co.nz/42336124/residential/rent/g06-246-khyber-pass-rd-newmarket
{'Address': '23 Paget Street, Freemans Bay, Auckland City', 'Price': '$2,450 per week', 'Info_1': 'House', 'Info_2': '3', 'Info_3': '3', 'Info_4': '1', 'Info_5': '', 'Info_6': '', 'I

Scraping: https://www.realestate.co.nz/42366991/residential/rent/33-139-quay-street-auckland-central
Scraping: https://www.realestate.co.nz/42430690/residential/rent/66b-queenstown-road-onehunga
Scraping: https://www.realestate.co.nz/42439743/residential/rent/3-exler-place-avondale
{'Address': '33/139 Quay Street, Auckland Central, Auckland City', 'Price': '$825 per week', 'Info_1': 'Apartment', 'Info_2': '1', 'Info_3': '1', 'Info_4': 'Furnished', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
Scraping: https://www.realestate.co.nz/42367499/residential/rent/1-51-aranui-road-mount-wellington
{'Address': '3 Exler Place, Avondale, Auckland City', 'Price': '$1,100 per week', 'Info_1': 'House', 'Info_2': '5', 'Info_3': '2', 'Info_4': '2', 'Info_5': '2', 'Info_6': '1 ensuite', 'Info_7': '', 'Info_8': ''}
{'Address': '1/51 Aranui Road, Mount Wellington, Auckland City', 'Price': '$950 per week', 'Info_1': 'Townhouse', 'Info_2': '4', 'Info_3': '3', 'Info_4': '3', 'Info_5': 'Non furnish

...

Scraping: https://www.realestate.co.nz/42416147/residential/rent/7-22-oak-street-royal-oak
{'Address': '7/22 Oak Street, Royal Oak, Auckland City', 'Price': '$320 per week', 'Info_1': 'House', 'Info_2': '1', 'Info_3': '1', 'Info_4': 'Non furnished', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
Scraping: https://www.realestate.co.nz/42436212/residential/rent/409-4-dockside-lane-auckland-central
{'Error': 'Message: timeout: Timed out receiving message from renderer: 247.218\n  (Session info: chrome=118.0.5993.70)\nStacktrace:\n0   chromedriver                        0x0000000100864510 chromedriver + 4310288\n1   chromedriver                        0x000000010085c4bc chromedriver + 4277436\n2   chromedriver                        0x000000010048fb6c chromedriver + 293740\n3   chromedriver                        0x0000000100477f54 chromedriver + 196436\n4   chromedriver                        0x0000000100477d60 chromedriver + 195936\n5   chromedriver                        0x0000

Scraping: https://www.realestate.co.nz/42436759/residential/rent/5d-18-cranbrook-place-glendowie
Scraping: https://www.realestate.co.nz/42418729/residential/rent/102-taniwha-street-waiotaiki-bay
{'Address': '5D/18 Cranbrook Place, Glendowie, Auckland City', 'Price': '$755 per week', 'Info_1': 'House', 'Info_2': '3', 'Info_3': '1', 'Info_4': '90m2', 'Info_5': '2', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Address': '102 Taniwha Street, Waiotaiki Bay, Auckland City', 'Price': '$600 per week', 'Info_1': 'House', 'Info_2': '2', 'Info_3': '1', 'Info_4': '1', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
Scraping: https://www.realestate.co.nz/42436849/residential/rent/52b-king-george-avenue-epsom
Scraping: https://www.realestate.co.nz/42418757/residential/rent/6-960-new-north-road-mount-albert
{'Address': '52B King George Avenue, Epsom, Auckland City', 'Price': '$1,400 per week', 'Info_1': 'House', 'Info_2': '5', 'Info_3': '3', 'Info_4': '2', 'Info_5': '1', 'Info_6': 'Non furnish

{'Address': 'a/1 Hoheria Road, Onehunga, Auckland City', 'Price': '$680 per week', 'Info_1': 'House', 'Info_2': '3', 'Info_3': '1', 'Info_4': '2', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
Scraping: https://www.realestate.co.nz/42421471/residential/rent/4h-19-fleet-street-eden-terrace
Scraping: https://www.realestate.co.nz/42437161/residential/rent/2205-32-swanson-street-auckland-central
{'Address': '4H/19 Fleet Street, Eden Terrace, Auckland City', 'Price': '$695 per week', 'Info_1': 'House', 'Info_2': '2', 'Info_3': '1', 'Info_4': '1', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Address': '2205/32 Swanson Street, Auckland Central, Auckland City', 'Price': '$460 per week', 'Info_1': 'Apartment', 'Info_2': '1', 'Info_3': '1', 'Info_4': 'Non furnished', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
Scraping: https://www.realestate.co.nz/42421608/residential/rent/104-2-finch-street-morningside
Scraping: https://www.realestate.co.nz/42437182/residenti

Scraping: https://www.realestate.co.nz/42425338/residential/rent/4-183-mt-albert-road-sandringham
{'Address': '19/1 Glenside Crescent, Auckland Central, Auckland City', 'Price': '$750 per week', 'Info_1': 'Townhouse', 'Info_2': '2', 'Info_3': '2', 'Info_4': 'Non furnished', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Address': '4/183 Mt Albert Road, Sandringham, Auckland City', 'Price': '$620 per week', 'Info_1': 'Unit', 'Info_2': '2', 'Info_3': '1', 'Info_4': '1', 'Info_5': '1', 'Info_6': 'Non furnished', 'Info_7': '', 'Info_8': ''}
Scraping: https://www.realestate.co.nz/42437997/residential/rent/3-296-ellerslie-panmure-highway-mount-wellington
Scraping: https://www.realestate.co.nz/42425389/residential/rent/6-245-gillies-avenue-epsom
{'Address': '3/296 Ellerslie Panmure Highway, Mount Wellington, Auckland City', 'Price': '$585 per week', 'Info_1': 'Unit', 'Info_2': '2', 'Info_3': '1', 'Info_4': '1', 'Info_5': 'Non furnished', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'

Scraping: https://www.realestate.co.nz/42438017/residential/rent/8i-508-queen-street-auckland-central
{'Address': '8I/508 Queen Street, Auckland Central, Auckland City', 'Price': '$400 per week', 'Info_1': 'Apartment', 'Info_2': '1', 'Info_3': '1', 'Info_4': 'Furnished', 'Info_5': '', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
Scraping: https://www.realestate.co.nz/42426148/residential/rent/1a-1-haverstock-sandringham
Scraping: https://www.realestate.co.nz/42438019/residential/rent/1-24-douglas-road-mount-eden
{'Address': '1A/1 Haverstock, Sandringham, Auckland City', 'Price': '$670 per week', 'Info_1': 'House', 'Info_2': '3', 'Info_3': '1', 'Info_4': '1', 'Info_5': '1', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Address': '1/24 Douglas Road, Mount Eden, Auckland City', 'Price': '$670 per week', 'Info_1': 'House', 'Info_2': '2', 'Info_3': '1', 'Info_4': '1', 'Info_5': 'Non furnished', 'Info_6': '', 'Info_7': '', 'Info_8': ''}
{'Error': 'Message: timeout: Timed out receiving message fro

In [79]:
url = "https://raw.githubusercontent.com/robertoaltran/Population/main/scraped_data.csv"

df = pd.read_csv(url)
df = df.dropna(subset=['Address'])

In [80]:
df.head(100)

,Address,Price,Info_1,Info_2,Info_3,Info_4,Info_5,Info_6,Info_7,Info_8,Error
0,"20/9A Esplanade Road, Mount Eden, Auckland City",$360 per week,Unit,1.0,1.0,1,NaN,NaN,NaN,NaN,NaN
1,"46 Montrose Street, Point Chevalier, Auckland ...",$650 per week,House,2.0,1.0,2,NaN,NaN,NaN,NaN,NaN
2,"1/351 Mount Eden Road, Mount Eden, Auckland City",$530 per week,House,2.0,1.0,NaN,NaN,NaN,1 Separate toilet,NaN,NaN
3,"64 School Road, Kingsland, Auckland City",$750 per week,House,3.0,1.0,2,NaN,NaN,NaN,NaN,NaN
4,"2E/25 Rutland Street, Auckland Central, Auckla...",$370 per week,Apartment,1.0,1.0,Non furnished,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
101,"PU17/36-38 Parry Road, Mount Wellington, Auckl...",$660 per week,Townhouse,2.0,1.0,1,Furnished,NaN,NaN,NaN,NaN
102,"102 Taniwha Street, Waiotaiki Bay, Auckland City",$600 per week,House,2.0,1.0,1,NaN,NaN,NaN,NaN,NaN
103,"6/960 New North Road, Mount Albert, Auckland City",$650 per week,Apartment,3.0,1.0,Non furnished,NaN,NaN,NaN,NaN,NaN
104,"GF/83 New North Road, Eden Terrace, Auckland City",$530 per week,Apartment,1.0,1.0,1,NaN,NaN,NaN,NaN,NaN


In [81]:
def is_numeric(value):
    #Check if the value is numeric.
    if pd.isna(value):
        return False
    try:
        int(value)
        return True
    except ValueError:
        return False

In [83]:
column_rename = {
    'Info_1': 'House Type',
    'Info_2': 'Bedroom',
    'Info_3': 'Bathroom',
    'Info_4': 'Garage',
    'Info_5': 'Other Feature 1',
    'Info_6': 'Other Feature 2',
}

df.rename(columns=column_rename, inplace=True)

# Remover as colunas "Info 8" e "Error"
df.drop(['Info_7', 'Info_8', 'Error'], axis=1, inplace=True)

In [88]:
# Convert 'Price' column to numerical by removing unwanted strings
df['Price'] = df['Price'].astype(str)

df['Price'] = df['Price'].str.replace(r'\$', '', regex=True)
df['Price'] = df['Price'].str.replace(r' per week', '', regex=True)
df['Price'] = df['Price'].str.replace(r',', '', regex=True)
#df['Price'] = pd.to_numeric(df['Price'], errors='coerce')  # Convert to numeric (float)

# Handle numeric values in 'Bedroom', 'Bathroom' and 'Garage'
df['Bedroom'] = df['Bedroom'].apply(lambda x: int(x) if is_numeric(x) else None)
df['Bathroom'] = df['Bathroom'].apply(lambda x: int(x) if is_numeric(x) else None)
mask_non_numeric_garage = ~df['Garage'].apply(is_numeric)
df.loc[mask_non_numeric_garage, ['Garage', 'Other Feature 1', 'Other Feature 2']] = df.loc[mask_non_numeric_garage, ['Other Feature 1', 'Garage', 'Other Feature 2']].values

# Convert the columns to Int64 type
for col in ['Bedroom', 'Bathroom', 'Garage']:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(pd.Int64Dtype())

# Handle cases where 'Other Feature 2' is numeric but 'Other Feature 1' isn't
mask = df['Other Feature 2'].apply(is_numeric) & ~df['Other Feature 1'].apply(is_numeric)
df.loc[mask, ['Other Feature 1', 'Other Feature 2']] = df.loc[mask, ['Other Feature 2', 'Other Feature 1']].values

# Add up numeric 'Garage' and 'Other Feature 1' values
mask = df['Garage'].apply(is_numeric) & df['Other Feature 1'].apply(is_numeric)
df.loc[mask, 'Garage'] += df.loc[mask, 'Other Feature 1'].astype(int)
df.loc[mask, 'Other Feature 1'] = df.loc[mask, 'Other Feature 2']
df.loc[mask, 'Other Feature 2'] = None

# Handle 'Furniture'
df['Furniture'] = 0
furnished_mask = df['Other Feature 1'].str.contains('Furnished', na=False) | df['Other Feature 2'].str.contains('Furnished', na=False)
df.loc[furnished_mask, 'Furniture'] = 1
df.loc[furnished_mask, ['Other Feature 1', 'Other Feature 2']] = None

non_furnished_mask = df['Other Feature 1'].str.contains('Non furnished', na=False) | df['Other Feature 2'].str.contains('Non furnished', na=False)
df.loc[non_furnished_mask, ['Other Feature 1', 'Other Feature 2']] = None

# Impute missing values in 'House Type'
df['House Type'].fillna('Unknown', inplace=True)

# One-hot encoding for 'House Type'
df = pd.get_dummies(df, columns=['House Type'], prefix='Type')

# Ensure all categories are present and if not, add them
categories = ["House", "Apartment", "Unit", "Townhouse", "Unknown"]
for cat in categories:
    col_name = 'Type_' + cat
    if col_name not in df.columns:
        df[col_name] = 0


In [89]:
df.head(10)

,Address,Price,Bedroom,Bathroom,Garage,Other Feature 1,Other Feature 2,Furniture,Type_Apartment,Type_Carpark,Type_House,Type_Studio,Type_Townhouse,Type_Unit,Type_Unknown
0,"20/9A Esplanade Road, Mount Eden, Auckland City",360.0,1,1,1,NaN,NaN,0,0,0,0,0,0,1,0
1,"46 Montrose Street, Point Chevalier, Auckland ...",650.0,2,1,2,NaN,NaN,0,0,0,1,0,0,0,0
2,"1/351 Mount Eden Road, Mount Eden, Auckland City",530.0,2,1,<NA>,NaN,NaN,0,0,0,1,0,0,0,0
3,"64 School Road, Kingsland, Auckland City",750.0,3,1,2,NaN,NaN,0,0,0,1,0,0,0,0
4,"2E/25 Rutland Street, Auckland Central, Auckla...",370.0,1,1,<NA>,None,None,0,1,0,0,0,0,0,0
6,"4G/16 Market Place Viaduct, Auckland Central, ...",520.0,1,1,<NA>,NaN,NaN,0,1,0,0,0,0,0,0
7,"29/139 Quay Street, Auckland Central, Auckland...",790.0,1,1,<NA>,None,None,1,1,0,0,0,0,0,0
8,"147 Quay Street, Auckland Central, Auckland City",780.0,1,1,<NA>,None,None,1,1,0,0,0,0,0,0
9,"174A White Swan Road, Mount Roskill, Auckland ...",695.0,3,2,3,None,None,0,0,0,1,0,0,0,0
10,"B3/15 Scanlan Street, Grey Lynn, Auckland City",750.0,2,1,1,None,None,0,0,0,0,0,1,0,0


In [90]:
df['Suburb'] = df['Address'].str.split(',').str[-2].str.strip()


In [91]:
df.head()

,Address,Price,Bedroom,Bathroom,Garage,Other Feature 1,Other Feature 2,Furniture,Type_Apartment,Type_Carpark,Type_House,Type_Studio,Type_Townhouse,Type_Unit,Type_Unknown,Suburb
0,"20/9A Esplanade Road, Mount Eden, Auckland City",360.0,1,1,1,NaN,NaN,0,0,0,0,0,0,1,0,Mount Eden
1,"46 Montrose Street, Point Chevalier, Auckland ...",650.0,2,1,2,NaN,NaN,0,0,0,1,0,0,0,0,Point Chevalier
2,"1/351 Mount Eden Road, Mount Eden, Auckland City",530.0,2,1,<NA>,NaN,NaN,0,0,0,1,0,0,0,0,Mount Eden
3,"64 School Road, Kingsland, Auckland City",750.0,3,1,2,NaN,NaN,0,0,0,1,0,0,0,0,Kingsland
4,"2E/25 Rutland Street, Auckland Central, Auckla...",370.0,1,1,<NA>,None,None,0,1,0,0,0,0,0,0,Auckland Central


In [61]:
# Obter valores únicos da coluna "Area_Description"
unique_values = df['Suburb'].unique()
# Filtrar apenas valores que são strings (excluindo NaN ou outros tipos)
string_values = [value for value in unique_values if isinstance(value, str)]

# Ordenar esses valores em ordem alfabética
sorted_unique_values = sorted(string_values)

# Imprimir os valores
for value in sorted_unique_values:
    print(value)

Auckland Central
Avondale
Blockhouse Bay
Eden Terrace
Ellerslie
Epsom
Freemans Bay
Glen Innes
Glendowie
Grafton
Greenlane
Grey Lynn
Herne Bay
Hillsborough
Kingsland
Kohimarama
Meadowbank
Mission Bay
Morningside
Mount Albert
Mount Eden
Mount Roskill
Mount Wellington
New Windsor
Newmarket
One Tree Hill
Onehunga
Otahuhu
Parnell
Point Chevalier
Point England
Ponsonby
Remuera
Royal Oak
Saint Heliers
Saint Johns
Saint Marys Bay
Sandringham
St Heliers
Stonefields
Three Kings
Waiotaiki Bay
Waterview
Westmere
